In [1]:
# 매개변수 최적화
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="./content/data_fashionmnist",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="./content/data_fashionmnist",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
# 하이퍼파라미터: 모델 최적화에 영향을 미칠 수 있는 조절 가능한 변수
# 에폭 수, 배치 크기, 학습률
learning_rate = 1e-3
batch_size = 64
epoch = 5

In [3]:
# 손실 함수 초기화
loss_fn = nn.CrossEntropyLoss()
loss_fn

CrossEntropyLoss()

In [4]:
# 옵티마이저: 최적화 알고리즘
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [5]:
# 학습 단계에서의 최적화 단계
# 1. optimizer.zero_grad()를 호출하여 모엘 매개변수의 변화도를 재설정
# 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기위해 반복할 때마다 명시적으로 0으로 설정
# 2. loss.backward()를 호출하여 예측 손실을 역전파
# 3. 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

In [6]:
#전체 구현
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    
    

In [7]:
# 손실 함수와 옵티마이저를 초기화하고 train_loop와 test_loop에 전달
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.310161  [    0/60000]
loss: 2.293371  [ 6400/60000]
loss: 2.277639  [12800/60000]
loss: 2.266681  [19200/60000]
loss: 2.259396  [25600/60000]
loss: 2.239500  [32000/60000]
loss: 2.228448  [38400/60000]
loss: 2.203548  [44800/60000]
loss: 2.196976  [51200/60000]
loss: 2.178453  [57600/60000]
Test Error: 
 Accuracy: 59.9%, Avg loss: 2.164812 

Epoch 2
-------------------------------
loss: 2.174962  [    0/60000]
loss: 2.159849  [ 6400/60000]
loss: 2.106599  [12800/60000]
loss: 2.117087  [19200/60000]
loss: 2.071634  [25600/60000]
loss: 2.026671  [32000/60000]
loss: 2.040063  [38400/60000]
loss: 1.968915  [44800/60000]
loss: 1.971690  [51200/60000]
loss: 1.911534  [57600/60000]
Test Error: 
 Accuracy: 60.6%, Avg loss: 1.900144 

Epoch 3
-------------------------------
loss: 1.933543  [    0/60000]
loss: 1.902332  [ 6400/60000]
loss: 1.786415  [12800/60000]
loss: 1.818282  [19200/60000]
loss: 1.719923  [25600/60000]
loss: 1.675566  [32000/600